## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_3 = pd.read_csv("../../바이너리_최종데이터_1004/gcc8/o3/o3_bincore8.csv", index_col=0)
print(bin8_3.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_3.reset_index(inplace=True, drop=True)

print('shape -', bin8_3.shape)
print('reset_index 완료')
print('input data shape')
bin8_3.head()

(31784379, 2)
shape - (31784379, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [12]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_3['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_3['label'].value_counts())

256
0    31747485
1       36894
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [13]:
idx_bin = bin8_3[bin8_3['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 16 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_3), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_3', len(ls_idx_bin))

# loc 로 수정필요
bin8_3_Ngram = bin8_3.loc[ls_idx_bin,:].copy()

590304
나머지 0
최종 길이 590304
bin8_3 590304


## (4) false data 만들기

In [14]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_3_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_3.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_3)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_3[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

59304.0
0 27387511
1000 16086943
2000 10746324
3000 13324932
4000 21547591
5000 24389343
6000 28624969
7000 10755937
8000 2140375
9000 10872346
10000 13412273
11000 20205603
12000 14447846
13000 23752932
14000 21309297
15000 8341823
16000 30664255
17000 3163102
18000 11897603
19000 2309165
20000 12932436
21000 23923892
22000 7594265
23000 15463833
24000 8435093
25000 8579234
26000 9901853
27000 16670892
28000 6762683
29000 8031949
30000 25981789
31000 1233433
32000 3385510
33000 14394919
34000 12302448
35000 15933774
36000 28934881
37000 6295957
38000 10081105
39000 10903722
40000 16856385
41000 7718005
42000 5575660
43000 17838951
44000 13488506
45000 3288155
46000 12035062
47000 469858
48000 14349919
49000 29289141
50000 6546206
51000 3914622
52000 3368837
53000 4698189
54000 31568258
55000 26149671
56000 24050514
57000 29166264
58000 5097320
59000 18799031
완료
59304


In [15]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_3['label'].value_counts()[1])

59304
36894


## (5) False Data + True Data 합치기

In [16]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_3_Ngram])
final.shape

(1897728, 2)

## (6) one hot encoding

In [17]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_3_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_3_onehot_Ngram = pd.concat([final['label'], bc8_3_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_3_onehot_Ngram.shape)

원핫인코딩완료
(1897728, 257)


In [18]:
# 훈련 데이터, 훈련 라벨
x_bc8_3 = bc8_3_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_3 = bc8_3_onehot_Ngram['label'].to_numpy()
print(x_bc8_3.shape, x_bc8_3.shape)

x_bc8_3 = x_bc8_3.reshape(-1, right_idx, x_bc8_3.shape[1])
y_bc8_3 = y_bc8_3.reshape(-1, right_idx, 1)

print(x_bc8_3.shape, y_bc8_3.shape)

(1897728, 256) (1897728, 256)
(118608, 16, 256) (118608, 16, 1)


In [19]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_3.shape[0])

x_bc8_3 = x_bc8_3[p]
y_bc8_3 = y_bc8_3[p]

print(x_bc8_3.shape, y_bc8_3.shape)

(118608, 16, 256) (118608, 16, 1)


## (7) 모델

In [20]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(24, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [21]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_3, y_bc8_3):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_3[train],
               y_bc8_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 106747 samples
Epoch 1/10
106747/106747 [==============================] - 59s 555us/sample - loss: 0.0231 - accuracy: 0.9935
Epoch 2/10
106747/106747 [==============================] - 48s 451us/sample - loss: 0.0084 - accuracy: 0.9974
Epoch 3/10
106747/106747 [==============================] - 59s 556us/sample - loss: 0.0068 - accuracy: 0.9979
Epo

accuracy_score 0.9987353511508305
recall_score 0.9665200391006843
precision_score 0.9746180384425825
f1_score 0.9705521472392638
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 106747 samples
Epoch 1/10
106747/106747 [==============================] - 89s 836us/sample - loss: 0.0034 - accuracy: 0.9990
Epoch 2/10
106747/106747 [==============================] - 72s 678us/sample - loss: 0.0032 - 

accuracy_score 0.9990620521035326
recall_score 0.9785344189489267
precision_score 0.9775696327335469
f1_score 0.9780517879161528
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 106747 samples
Epoch 1/10
106747/106747 [==============================] - 98s 914us/sample - loss: 0.0022 - accuracy: 0.9994
Epoch 2/10
106747/106747 [==============================] - 60s 561us/sample - loss: 0.0021 - 

accuracy_score 0.9992254025798837
recall_score 0.9836065573770492
precision_score 0.9815078236130867
f1_score 0.9825560697757209
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 106747 samples
Epoch 1/10
106747/106747 [==============================] - 74s 697us/sample - loss: 0.0017 - accuracy: 0.9995
Epoch 2/10
106747/106747 [==============================] - 72s 672us/sample - loss: 0.0017 - 

accuracy_score 0.9992886350223421
recall_score 0.9820374015748031
precision_score 0.9847026893659018
f1_score 0.9833682394973512
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 106747 samples
Epoch 1/10
106747/106747 [==============================] - 91s 849us/sample - loss: 0.0014 - accuracy: 0.9996
Epoch 2/10
106747/106747 [==============================] - 70s 655us/sample - loss: 0.0013 - 

...

accuracy_score 0.9992675575415226
recall_score 0.9778325123152709
precision_score 0.9878079124160238
f1_score 0.9827949003589553
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 106747 samples
Epoch 1/10
106747/106747 [==============================] - 67s 631us/sample - loss: 0.0011 - accuracy: 0.9997
Epoch 2/10
106747/106747 [==============================] - 56s 520us/sample - loss: 0.0010 - 

accuracy_score 0.9993676755754153
recall_score 0.9806146572104019
precision_score 0.9909221213569039
f1_score 0.9857414448669202
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 106747 samples
Epoch 1/10
106747/106747 [==============================] - 72s 676us/sample - loss: 9.5185e-04 - accuracy: 0.9997
Epoch 2/10
106747/106747 [==============================] - 48s 448us/sample - loss: 8.805

accuracy_score 0.9994730629795127
recall_score 0.9895478852698104
precision_score 0.9861918604651163
f1_score 0.9878670225673382
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 106748 samples
Epoch 1/10
106748/106748 [==============================] - 81s 760us/sample - loss: 8.0375e-04 - accuracy: 0.9998
Epoch 2/10
106748/106748 [==============================] - 58s 540us/sample - loss: 7.330

accuracy_score 0.999552065767285
recall_score 0.9902605308010713
precision_score 0.9890564202334631
f1_score 0.989658109259034
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 106748 samples
Epoch 1/10
106748/106748 [==============================] - 81s 757us/sample - loss: 7.1878e-04 - accuracy: 0.9998
Epoch 2/10
106748/106748 [==============================] - 66s 622us/sample - loss: 6.3571e

accuracy_score 0.9996363827993254
recall_score 0.9927572427572428
precision_score 0.9900373599003736
f1_score 0.9913954358398802

K-fold cross validation Accuracy: [0.9987353511508305, 0.9990620521035326, 0.9992254025798837, 0.9992886350223421, 0.9991885169884496, 0.9992675575415226, 0.9993676755754153, 0.9994730629795127, 0.999552065767285, 0.9996363827993254]

K-fold cross validation Recall: [0.9665200391006843, 0.9785344189489267, 0.9836065573770492, 0.9820374015748031, 0.977363184079602, 0.9778325123152709, 0.9806146572104019, 0.9895478852698104, 0.9902605308010713, 0.9927572427572428]

K-fold cross validation Precision: [0.9746180384425825, 0.9775696327335469, 0.9815078236130867, 0.9847026893659018, 0.9842184368737475, 0.9878079124160238, 0.9909221213569039, 0.9861918604651163, 0.9890564202334631, 0.9900373599003736]

K-fold cross validation F1-Score: [0.9705521472392638, 0.9780517879161528, 0.9825560697757209, 0.9833682394973512, 0.9807788317523715, 0.9827949003589553, 0.98574144

## (9) 평가지표

In [22]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.99927967025081
10-Fold Cross_validation. Recall : 0.9819074429434863
10-Fold Cross_validation. Precision : 0.9846632295400747
10-Fold Cross_validation. F1-Score : 0.9832763989072989


In [23]:
model1.save('gcc8_bin_core_s16_h24_o3.h5')
print('save 완료')

save 완료
